In [1]:
%load_ext autoreload
%autoreload 2

In [30]:
import sys
import os
from os.path import join
sys.path.append(os.pardir)

In [63]:
from train.train_daily_gan import Model
from utils.utils import make_output_folder

In [17]:
from data.dataset import bird_dataset_single_hdf

In [14]:
import random
import gc

In [5]:
import torch
import numpy as np

In [ ]:
opts

In [57]:
opts['ngf'] = 96
opts['ndf'] =96
opts['train_residual'] = True
opts['batchSize'] = 64
opts['lr'] = 1e-4
opts['nepochs'] = 10
opts['init_params'] = 'kmeans'
opts['manualSeed'] = 90
opts['nz'] = 12
opts['outf'] = '/home/songbird/datapartition/mdgan_output/daily_gan/r15y5_nz12/'
opts['datapath'] = '/home/songbird/datapartition/all_birds.hdf'

In [58]:
opts['cuda'] = True

In [59]:
# fix seed
print("Random Seed: ", opts['manualSeed'])
random.seed(opts['manualSeed'])
torch.manual_seed(opts['manualSeed'])
if opts['cuda']:
    torch.cuda.manual_seed(opts['manualSeed'])
opts['hmm_random_state'] = opts['manualSeed']



Random Seed:  90


# make dataset object

In [60]:
opts['birdname'] = 'r15y5'

In [28]:
# make bird dataset
dataset = bird_dataset_single_hdf(opts['datapath'], opts['birdname'])
# how many days are there ? 
ndays = dataset.ndays

    

... total number of folders for bird r15y5 = 41 ...


In [34]:
len(traindataloader)

880

# build initial model object

In [64]:
# make model
model = Model(dataset, '', opts, None)
# make a save folder (run e.g. 2021-09-10_15_12_31)
opts['outf'] = make_output_folder(opts['outf'])

Using L1 loss


In [ ]:


for day in range(ndays):
    print('\n\n.... ### WORKING ON DAY %d for bird %s ### .....'%(day, opts['birdname']))

    # update output folder
    outpath = join(opts['outf'], 'day_' + str(day))
    os.makedirs(outpath, exist_ok=True)
    model.outpath = outpath

    # re-initialize networks
    if day > 0 and not opts['chain_networks']:
        model._init_networks()

    # make traindataloader
    #traindataloader = model.make_dataloader(day)
    # train networks
    model.train_one_day(day, traindataloader)

    # then, train hmm
    for k in range(opts['hidden_state_size']):

        model.train_hmm(day, opts['hidden_state_size'][k])

    # clear the saved spectrogram and latent vectors arrays
    model.X = None 
    model.Z = None

In [66]:
del model
gc.collect()

3919